In [1]:
import os
import polars as pl
import altair as alt

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [31]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [45]:
def podil_vyskytu_behem_roku(hledani, popisek):
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("pocet_vyskytu")
    ).group_by(
        pl.col("datum").dt.ordinal_day()
    ).agg(
        pl.col("pocet_vyskytu").sum()
    ).join(
        df.group_by(
            pl.col("datum").dt.ordinal_day()
        ).agg(
            pl.col("pocet_slov").sum()
        ), on="datum", how="right"
    ).sort(
        by="datum"
    ).with_columns(
        (pl.col("pocet_vyskytu") / pl.col("pocet_slov")).alias("pomer")
    )

In [43]:
podil_vyskytu_behem_roku(r"(?i)váno[cč]","Vánoce")

pocet_vyskytu,datum,pocet_slov,podil
u32,i16,u32,f64
0,1,3804,0.0
10,5,98574,0.000101
0,6,2999,0.0
1,7,3955,0.000253
5,8,57702,0.000087
…,…,…,…
8,358,16093,0.000497
0,361,4435,0.0
4,362,7624,0.000525


In [49]:
graf(podil_vyskytu_behem_roku(r"(?i)váno[cč]","Vánoce").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [53]:
graf(podil_vyskytu_behem_roku(r"(?i)velikono[cč]","Velikonoce").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [57]:
graf(podil_vyskytu_behem_roku(r"Masaryk","Masaryk").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [61]:
graf(podil_vyskytu_behem_roku(r"(?i)sex","sex").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [65]:
graf(podil_vyskytu_behem_roku(r"(?i)hokej","hokej").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [67]:
graf(podil_vyskytu_behem_roku(r"(?i)cyklist","cyklistika").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [69]:
graf(podil_vyskytu_behem_roku(r"(?i)ukrajin","Ukrajina").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [71]:
graf(podil_vyskytu_behem_roku(r"(?i)pyrotechnik","pyrotechnika").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [73]:
graf(podil_vyskytu_behem_roku(r"(?i)jaro","jaro").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [75]:
graf(podil_vyskytu_behem_roku(r"(?i)léto","léto").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [79]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]skliz[en]","sklizeň").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [81]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]pivo","pivo").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [83]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]advent","advent").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [85]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]předsevzetí","předsevzetí").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [87]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]děti","děti").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [89]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]smrt","smrt").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [91]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]osvětlení","osvětlení").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [93]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]horko","horko").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [97]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]povod[en]","povodně").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [99]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]dušičk","dušičky").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [101]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]prázdnin","prázdnin").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [109]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]štědr\w{0,5}\s{0,1}(večer|den)","").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)

In [111]:
graf(podil_vyskytu_behem_roku(r"(?i)[^\w]koledy","koledy").filter(pl.col("pocet_slov") > 10000))

alt.Chart(...)